#T1

In [1]:
r = -5 #live-in reward
R1 = 10 #exit reward
R2 = -5 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward = {(col, row):r for col in range(1,6) for row in range(1,6)}
U[D]= R1
U[H]= R2
U_new[D]= R1
U_new[H]= R2
reward[D]= R1
reward[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U[i] = None
  U_new[i] = None
  reward[i] = None
print(U)
print(U_new)
print(reward)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -5, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 10, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -5, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 10, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -5, (1, 3): -5, (1, 4): -5, (1, 5): None, (2, 1): -5, (2, 2): -5, (2, 3): -5, (2, 4): -5, (2, 5): -5, (3, 1): -5, (3, 2): -5, (3, 3): -5, (3, 4): -5, (3, 5): -5, (4, 1): -5, (4, 2): -5, (4, 3): -5, (4, 4): -5, (4, 5): 10, (5, 1): None, (5, 2): -5, (5, 3): -5, (5, 4): -5, (5, 5): None}


In [2]:
def update_directions(directions,obstacle,col_size,row_size,curr_state):
  for i in range(len(directions)):
    if (directions[i] in obstacle) or (directions[i][0] > col_size or directions[i][1] > row_size) or (directions[i][0] < 1 or directions[i][1] < 1):
      directions[i] = curr_state
  return directions

def pattern_order(destination=D, obstacle1=obstacle, col_size=5, row_size=5, hazard=H):
  visited_nodes = []
  curr_child_nodes = [destination] #curr iteration child nodes
  child_nodes_order = [] #list of list i.e [[child_nodes in iteration 1], [child_nodes in iteration 2], .....]
  while curr_child_nodes:
    nxt_list = [] #next list of next child nodes to be iterated in the next while loop
    for child in curr_child_nodes:
      col, row = child
      next_children = [(col+1,row),(col, row+1),(col-1, row),(col, row-1)]
      # dir = set(update_directions(next_children,obstacle1,col_size,row_size,(col,row)))
      # #to remove the None (because of obstacles)
      # dir.remove(None)
      possible_nxt_child = list(set(update_directions(next_children,obstacle1,col_size,row_size,(col,row))))
      for node in possible_nxt_child:
        nxt_list.append(node) if (node not in visited_nodes and node not in [destination, hazard]) else nxt_list
        visited_nodes.append(node) if (node not in visited_nodes and node not in [destination,hazard]) else visited_nodes
    child_nodes_order.append(nxt_list)
    curr_child_nodes = nxt_list
  return visited_nodes, child_nodes_order

In [3]:
u, v = pattern_order(D,obstacle,5,5,H)
print(u)
print(v)

[(4, 4), (3, 5), (5, 4), (3, 4), (4, 3), (2, 5), (5, 3), (2, 4), (4, 2), (5, 2), (2, 3), (1, 4), (3, 2), (4, 1), (1, 3), (2, 2), (3, 1), (1, 2), (2, 1)]
[[(4, 4), (3, 5)], [(5, 4), (3, 4), (4, 3), (2, 5)], [(5, 3), (2, 4), (4, 2)], [(5, 2), (2, 3), (1, 4), (3, 2), (4, 1)], [(1, 3), (2, 2), (3, 1)], [(1, 2), (2, 1)], []]


In [4]:
def TransProb(state, U, col_size, row_size, obstacle, action):
  col, row = state
  new_directions = [(col+1,row),(col, row+1),(col-1, row),(col, row-1)] #right, up, left, down
  obstacle_idx = set()
  for i in range(len(new_directions)):
    if new_directions[i] in obstacle:
      obstacle_idx.add(i)
  directions = update_directions(new_directions,obstacle,col_size,row_size,(col,row))
  curr_right,curr_up,curr_left,curr_down  = directions[0], directions[1], directions[2], directions[3]
  n_dest = 0
  non_dest_index = [] #to save the obstacle positions index
  for i in range(len(directions)):
    if i not in obstacle_idx: #if not obstacle then add
      n_dest += 1
    else:
      non_dest_index.append(i)
  count = n_dest-1 #non-obstacle
  p1 = 0.9
  up_u, down_u, right_u, left_u = U[curr_up], U[curr_down], U[curr_right], U[curr_left]
  if action == 'up':
    if directions[3] != (col,row): #if bottom is not an obstacle but it is 180 degree it will become blocked so we decrease the count if bottom is not blocked.Since only obstacles are included earlier in n_dest
      count -= 1
    p2, p3 = 0.1/count, 0.1/count
    for index in non_dest_index:
      if index == 0:
        right_u = 0
      elif index == 1:
        up_u = 0
      elif index == 2:
        left_u = 0
    val = p1 * up_u + p2 * right_u + p3 * left_u
    return count, val
  elif action == 'down':
    if directions[1] != (col,row): #opposite direction is up
      count -= 1
    p2, p3 = 0.1/count, 0.1/count
    for index in non_dest_index:
      if index == 0:
        right_u = 0
      elif index == 3:
        down_u = 0
      elif index == 2:
        left_u = 0
    val = p1 * down_u + p2 * right_u + p3 * left_u
    return count, val
  elif action == 'left':
    if directions[0] != (col,row): #opposite direction is right
      count -= 1
    p2, p3 = 0.1/count, 0.1/count
    for index in non_dest_index:
      if index == 1:
        up_u = 0
      elif index == 3:
        down_u = 0
      elif index == 2:
        left_u = 0
    val = p1 * left_u + p2 * down_u + p3 * up_u
    return count, val
  elif action == 'right': #opoosite direction is left
    if directions[2] != (col,row):
      count -= 1
    p2, p3 = 0.1/count, 0.1/count
    for index in non_dest_index:
      if index == 0:
        right_u = 0
      elif index == 3:
        down_u = 0
      elif index == 1:
        up_u = 0
    val = p1 * right_u + p2 * up_u + p3 * down_u
    return count, val



In [5]:
import copy
# col_size, row_size = 5,5
def valueIteration(threshold_value=0.01, max_iter=9, gamma=0.9, reward=reward, U=U, obstacle1=obstacle, U_new=U_new,col_size=5, row_size=5, destination=D, hazard=H):
    curr_seen = seen = [destination]
    k = 1
    not_threshold_condition = True
    all_states_order, child_nodes_order = pattern_order(destination,obstacle1,col_size,row_size,hazard)
    while not_threshold_condition and k < max_iter:
        for state in all_states_order:
            max_val = float('-inf')
            for action in ['right', 'up', 'left', 'down']:
              demoni, Prob = TransProb(state, U, col_size, row_size, obstacle, action)
              max_val = max(max_val, Prob)
            U_new[state] = round(reward[state] + gamma * max_val,2)
            not_threshold_condition = (not_threshold_condition and (U_new[state] - U[state] < threshold_value))
        not_threshold_condition = not (not_threshold_condition)
        U = copy.deepcopy(U_new)
        k += 1
    print(f"At what iteration it's stopping {k}")
    return U

In [6]:
# u1 = valueIteration(0.01, 1, 0.9,reward, U, obstacle, U_new, 5, 5,D,H)
# u1

At what iteration it's stopping 1


{(1, 1): None,
 (1, 2): 0,
 (1, 3): 0,
 (1, 4): 0,
 (1, 5): None,
 (2, 1): 0,
 (2, 2): 0,
 (2, 3): 0,
 (2, 4): 0,
 (2, 5): 0,
 (3, 1): 0,
 (3, 2): 0,
 (3, 3): -5,
 (3, 4): 0,
 (3, 5): 0,
 (4, 1): 0,
 (4, 2): 0,
 (4, 3): 0,
 (4, 4): 0,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): 0,
 (5, 3): 0,
 (5, 4): 0,
 (5, 5): None}

In [ ]:
# u2 = valueIteration(0.01, 2, 0.9,reward, U, obstacle, U_new, 5, 5,D,H)
# u2

At what iteration it's stopping 2


{(1, 1): None,
 (1, 2): -5.0,
 (1, 3): -5.0,
 (1, 4): -5.0,
 (1, 5): None,
 (2, 1): -5.0,
 (2, 2): -5.0,
 (2, 3): -5.0,
 (2, 4): -5.0,
 (2, 5): -5.0,
 (3, 1): -5.0,
 (3, 2): -5.0,
 (3, 3): -5,
 (3, 4): -5.0,
 (3, 5): 3.1,
 (4, 1): -5.0,
 (4, 2): -5.0,
 (4, 3): -5.0,
 (4, 4): 3.1,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): -5.0,
 (5, 3): -5.0,
 (5, 4): -5.0,
 (5, 5): None}

In [ ]:
# u3 = valueIteration(0.01, 3, 0.9,reward, U, obstacle, U_new, 5, 5,D,H)
# u3

At what iteration it's stopping 3


{(1, 1): None,
 (1, 2): -5.9,
 (1, 3): -9.35,
 (1, 4): -5.9,
 (1, 5): None,
 (2, 1): -5.9,
 (2, 2): -9.5,
 (2, 3): -9.5,
 (2, 4): -9.5,
 (2, 5): -2.94,
 (3, 1): -9.35,
 (3, 2): -9.5,
 (3, 3): -5,
 (3, 4): -2.57,
 (3, 5): 3.01,
 (4, 1): -5.9,
 (4, 2): -9.5,
 (4, 3): -2.94,
 (4, 4): 2.65,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): -5.9,
 (5, 3): -9.35,
 (5, 4): -2.71,
 (5, 5): None}

In [ ]:
# u8 = valueIteration(0.01, 8, 0.9,reward, U, obstacle, U_new, 5, 5,D,H)
# u8

At what iteration it's stopping 8


{(1, 1): None,
 (1, 2): -6.59,
 (1, 3): -11.01,
 (1, 4): -6.26,
 (1, 5): None,
 (2, 1): -6.59,
 (2, 2): -11.07,
 (2, 3): -9.9,
 (2, 4): -7.76,
 (2, 5): -2.96,
 (3, 1): -11.06,
 (3, 2): -9.93,
 (3, 3): -5,
 (3, 4): -2.69,
 (3, 5): 3.12,
 (4, 1): -6.33,
 (4, 2): -8.39,
 (4, 3): -3.28,
 (4, 4): 2.84,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): -6.33,
 (5, 3): -7.94,
 (5, 4): -3.06,
 (5, 5): None}

In [6]:
u100 = valueIteration(0.01, 100, 0.9,reward, U, obstacle, U_new, 5, 5,D,H)
u100

At what iteration it's stopping 9


{(1, 1): None,
 (1, 2): -6.59,
 (1, 3): -11.01,
 (1, 4): -6.26,
 (1, 5): None,
 (2, 1): -6.59,
 (2, 2): -11.08,
 (2, 3): -9.9,
 (2, 4): -7.76,
 (2, 5): -2.96,
 (3, 1): -11.07,
 (3, 2): -9.93,
 (3, 3): -5,
 (3, 4): -2.69,
 (3, 5): 3.12,
 (4, 1): -6.32,
 (4, 2): -8.39,
 (4, 3): -3.28,
 (4, 4): 2.84,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): -6.32,
 (5, 3): -7.94,
 (5, 4): -3.06,
 (5, 5): None}

In [7]:
!pip install tabulate

In [8]:
from tabulate import tabulate
def visualise_table(row_size=5,col_size=5,input_val=u100):
  table = []
  for row in range(row_size,0,-1):
    new_table_value = []
    for col in range(1,col_size+1):
      # if input_val[(col,row)] == 0:
      #   input_val[(col,row)] = 0
      if input_val[(col,row)] == None:
        input_val[(col,row)] = 'X'
      new_table_value.append(input_val[(col,row)])
    table.append(new_table_value)
  return table, tabulate(table, tablefmt="fancy_grid")


In [9]:
u_tba, v = visualise_table(5,5,u100)
print(u_tba)
print(v)

[['X', -2.96, 3.12, 10, 'X'], [-6.26, -7.76, -2.69, 2.84, -3.06], [-11.01, -9.9, -5, -3.28, -7.94], [-6.59, -11.08, -9.93, -8.39, -6.32], ['X', -6.59, -11.07, -6.32, 'X']]
╒════════╤════════╤════════╤═══════╤═══════╕
│ X      │  -2.96 │   3.12 │ 10    │ X     │
├────────┼────────┼────────┼───────┼───────┤
│ -6.26  │  -7.76 │  -2.69 │  2.84 │ -3.06 │
├────────┼────────┼────────┼───────┼───────┤
│ -11.01 │  -9.9  │  -5    │ -3.28 │ -7.94 │
├────────┼────────┼────────┼───────┼───────┤
│ -6.59  │ -11.08 │  -9.93 │ -8.39 │ -6.32 │
├────────┼────────┼────────┼───────┼───────┤
│ X      │  -6.59 │ -11.07 │ -6.32 │ X     │
╘════════╧════════╧════════╧═══════╧═══════╛


In [11]:
def get_arrow(row, col, table, obstacle, destination, hazard):
  directions = [(row,col+1),(row+1,col),(row,col-1),(row-1,col)] #right, down, left, up
  row_size,col_size = len(table), len(table[0])
  for i in range(len(directions)):
    row_val, col_val = directions[i][0], directions[i][1]
    if (row_val >= row_size or col_val >= col_size) or (row_val < 0 or col_val < 0):
        directions[i] = (row, col)
    elif (row_val < row_size and col_val < col_size) and (row_val >= 0 and col_val >= 0):
      if (table[row_val][col_val] == 'X'):
        directions[i] = (row, col)
  values = []
  for direction in directions:
    if direction:
      row_val, col_val = direction
      values.append(table[row_val][col_val])
    else:
      values.append(float('-inf'))
  max_value = max(values)
  index_max_val = [i for i, value in enumerate(values) if value == max_value]
  ans = ''
  if max_value != table[row][col]:
    if max_value > table[row][col]:
      for index_val in index_max_val:
        if index_val == 0: #right
          ans += '→'
        elif index_val == 1: #down
          ans += '↓'
        elif index_val == 2: #left
          ans += '←'
        elif index_val == 3: #up
          ans += '↑'
    else:
      ans += '-'
  elif max_value == table[row][col]:
      for index_val in index_max_val:
        if index_val == 0: #right
          ans += '→'
        elif index_val == 1: #down
          ans += '↓'
        elif index_val == 2: #left
          ans += '←'
        elif index_val == 3: #up
          ans += '↑'
  return ans

def visualise_policy(table=u_tba,destination=D,hazard=H,obstacle1=obstacle):
  arrow = []
  for row in range(len(table)):
    new_arrow = []
    for col in range(len(table[row])):
      if (col+1,len(table)-row) != destination and (col+1,len(table)-row) != hazard and ((col+1,len(table)-row) not in obstacle1):
        arrow_direction = get_arrow(row,col,table,obstacle1,destination,hazard)
      else:
        arrow_direction = table[row][col]
      new_arrow.append(arrow_direction)
    arrow.append(new_arrow)
  return tabulate(arrow, tablefmt="fancy_grid")


In [12]:
policy_1 = visualise_policy(u_tba, D, H, obstacle)
print(policy_1)

╒════╤════╤════╤════╤════╕
│ X  │ →  │ →  │ 10 │ X  │
├────┼────┼────┼────┼────┤
│ ←↑ │ →  │ ↑  │ ↑  │ ←  │
├────┼────┼────┼────┼────┤
│ ↑  │ →  │ -5 │ ↑  │ ↑  │
├────┼────┼────┼────┼────┤
│ ↓← │ ↓← │ ↑  │ ↑  │ →↓ │
├────┼────┼────┼────┼────┤
│ X  │ ↓← │ →  │ →↓ │ X  │
╘════╧════╧════╧════╧════╛


#T3-1

In [13]:
# R1=50, R2=-50 and r=-5
r = -5 #live-in reward
R1 = 50 #exit reward
R2 = -50 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U_2 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new2 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward_2 = {(col, row):r for col in range(1,6) for row in range(1,6)}
U_2[D]= R1
U_2[H]= R2
U_new2[D]= R1
U_new2[H]= R2
reward_2[D]= R1
reward_2[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U_2[i] = None
  U_new2[i] = None
  reward_2[i] = None
print(U_2)
print(U_new2)
print(reward_2)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -50, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 50, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -50, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 50, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -5, (1, 3): -5, (1, 4): -5, (1, 5): None, (2, 1): -5, (2, 2): -5, (2, 3): -5, (2, 4): -5, (2, 5): -5, (3, 1): -5, (3, 2): -5, (3, 3): -50, (3, 4): -5, (3, 5): -5, (4, 1): -5, (4, 2): -5, (4, 3): -5, (4, 4): -5, (4, 5): 50, (5, 1): None, (5, 2): -5, (5, 3): -5, (5, 4): -5, (5, 5): None}


In [14]:
u100_2 = valueIteration(0.01, 100, 0.9,reward_2, U_2, obstacle, U_new2, 5, 5,D,H)
u100_2

At what iteration it's stopping 16


{(1, 1): None,
 (1, 2): -0.95,
 (1, 3): 4.91,
 (1, 4): 11.45,
 (1, 5): None,
 (2, 1): -2.79,
 (2, 2): 2.65,
 (2, 3): 9.2,
 (2, 4): 20.04,
 (2, 5): 28.39,
 (3, 1): 1.46,
 (3, 2): 5.35,
 (3, 3): -50,
 (3, 4): 28.82,
 (3, 5): 38.53,
 (4, 1): 7.6,
 (4, 2): 15.47,
 (4, 3): 24.35,
 (4, 4): 37.99,
 (4, 5): 50,
 (5, 1): None,
 (5, 2): 11.16,
 (5, 3): 18.47,
 (5, 4): 26.6,
 (5, 5): None}

In [15]:
u_tba, v = visualise_table(5,5,u100_2)
print(u_tba)
print(v)

[['X', 28.39, 38.53, 50, 'X'], [11.45, 20.04, 28.82, 37.99, 26.6], [4.91, 9.2, -50, 24.35, 18.47], [-0.95, 2.65, 5.35, 15.47, 11.16], ['X', -2.79, 1.46, 7.6, 'X']]
╒═══════╤═══════╤════════╤═══════╤═══════╕
│ X     │ 28.39 │  38.53 │ 50    │ X     │
├───────┼───────┼────────┼───────┼───────┤
│ 11.45 │ 20.04 │  28.82 │ 37.99 │ 26.6  │
├───────┼───────┼────────┼───────┼───────┤
│ 4.91  │  9.2  │ -50    │ 24.35 │ 18.47 │
├───────┼───────┼────────┼───────┼───────┤
│ -0.95 │  2.65 │   5.35 │ 15.47 │ 11.16 │
├───────┼───────┼────────┼───────┼───────┤
│ X     │ -2.79 │   1.46 │  7.6  │ X     │
╘═══════╧═══════╧════════╧═══════╧═══════╛


In [16]:
policy_2 = visualise_policy(u_tba, D, H, obstacle)
print(policy_2)

╒═══╤═══╤═════╤════╤═══╕
│ X │ → │ →   │ 50 │ X │
├───┼───┼─────┼────┼───┤
│ → │ → │ ↑   │ ↑  │ ← │
├───┼───┼─────┼────┼───┤
│ ↑ │ ↑ │ -50 │ ↑  │ ↑ │
├───┼───┼─────┼────┼───┤
│ ↑ │ ↑ │ →   │ ↑  │ ↑ │
├───┼───┼─────┼────┼───┤
│ X │ ↑ │ →   │ ↑  │ X │
╘═══╧═══╧═════╧════╧═══╛


#T3-2

In [17]:
# R1=100, R2=-500, r=-5
r = -5 #live-in reward
R1 = 100 #exit reward
R2 = -500 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U_3 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new3 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward_3 = {(col, row):r for col in range(1,6) for row in range(1,6)}
U_3[D]= R1
U_3[H]= R2
U_new3[D]= R1
U_new3[H]= R2
reward_3[D]= R1
reward_3[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U_3[i] = None
  U_new3[i] = None
  reward_3[i] = None
print(U_3)
print(U_new3)
print(reward_3)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -500, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 100, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -500, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 100, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -5, (1, 3): -5, (1, 4): -5, (1, 5): None, (2, 1): -5, (2, 2): -5, (2, 3): -5, (2, 4): -5, (2, 5): -5, (3, 1): -5, (3, 2): -5, (3, 3): -500, (3, 4): -5, (3, 5): -5, (4, 1): -5, (4, 2): -5, (4, 3): -5, (4, 4): -5, (4, 5): 100, (5, 1): None, (5, 2): -5, (5, 3): -5, (5, 4): -5, (5, 5): None}


In [18]:
u100_3 = valueIteration(0.01, 100, 0.9,reward_3, U_3, obstacle, U_new3, 5, 5,D,H)
u100_3

At what iteration it's stopping 18


{(1, 1): None,
 (1, 2): 21.13,
 (1, 3): 30.27,
 (1, 4): 40.6,
 (1, 5): None,
 (2, 1): 7.37,
 (2, 2): 14.59,
 (2, 3): 22.63,
 (2, 4): 54.62,
 (2, 5): 67.56,
 (3, 1): 12.2,
 (3, 2): 6.9,
 (3, 3): -500,
 (3, 4): 68.2,
 (3, 5): 82.79,
 (4, 1): 20.17,
 (4, 2): 30.4,
 (4, 3): 41.14,
 (4, 4): 81.93,
 (4, 5): 100,
 (5, 1): None,
 (5, 2): 39.18,
 (5, 3): 50.68,
 (5, 4): 63.64,
 (5, 5): None}

In [19]:
u_tba, v = visualise_table(5,5,u100_3)
print(u_tba)
print(v)

[['X', 67.56, 82.79, 100, 'X'], [40.6, 54.62, 68.2, 81.93, 63.64], [30.27, 22.63, -500, 41.14, 50.68], [21.13, 14.59, 6.9, 30.4, 39.18], ['X', 7.37, 12.2, 20.17, 'X']]
╒═══════╤═══════╤═════════╤════════╤═══════╕
│ X     │ 67.56 │   82.79 │ 100    │ X     │
├───────┼───────┼─────────┼────────┼───────┤
│ 40.6  │ 54.62 │   68.2  │  81.93 │ 63.64 │
├───────┼───────┼─────────┼────────┼───────┤
│ 30.27 │ 22.63 │ -500    │  41.14 │ 50.68 │
├───────┼───────┼─────────┼────────┼───────┤
│ 21.13 │ 14.59 │    6.9  │  30.4  │ 39.18 │
├───────┼───────┼─────────┼────────┼───────┤
│ X     │  7.37 │   12.2  │  20.17 │ X     │
╘═══════╧═══════╧═════════╧════════╧═══════╛


In [20]:
policy_3 = visualise_policy(u_tba, D, H, obstacle)
print(policy_3)

╒═══╤═══╤══════╤═════╤═══╕
│ X │ → │ →    │ 100 │ X │
├───┼───┼──────┼─────┼───┤
│ → │ → │ ↑    │ ↑   │ ← │
├───┼───┼──────┼─────┼───┤
│ ↑ │ ↑ │ -500 │ ↑   │ ↑ │
├───┼───┼──────┼─────┼───┤
│ ↑ │ ↑ │ →    │ ↑   │ ↑ │
├───┼───┼──────┼─────┼───┤
│ X │ ↑ │ →    │ ↑   │ X │
╘═══╧═══╧══════╧═════╧═══╛


#T3-3

using R1 and R2 from policy 3

In [21]:
# R1=100, R2=-500, r=-1
r = -1 #live-in reward
R1 = 100 #exit reward
R2 = -500 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U_4_1 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new4_1 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward_4_1 = {(col, row):r for col in range(1,6) for row in range(1,6)}
U_4_1[D]= R1
U_4_1[H]= R2
U_new4_1[D]= R1
U_new4_1[H]= R2
reward_4_1[D]= R1
reward_4_1[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U_4_1[i] = None
  U_new4_1[i] = None
  reward_4_1[i] = None
print(U_4_1)
print(U_new4_1)
print(reward_4_1)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -500, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 100, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -500, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 100, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -1, (1, 3): -1, (1, 4): -1, (1, 5): None, (2, 1): -1, (2, 2): -1, (2, 3): -1, (2, 4): -1, (2, 5): -1, (3, 1): -1, (3, 2): -1, (3, 3): -500, (3, 4): -1, (3, 5): -1, (4, 1): -1, (4, 2): -1, (4, 3): -1, (4, 4): -1, (4, 5): 100, (5, 1): None, (5, 2): -1, (5, 3): -1, (5, 4): -1, (5, 5): None}


In [22]:
u100_4_1 = valueIteration(0.01, 100, 0.9,reward_4_1, U_4_1, obstacle, U_new4_1, 5, 5,D,H)
u100_4_1

At what iteration it's stopping 20


{(1, 1): None,
 (1, 2): 41.14,
 (1, 3): 47.71,
 (1, 4): 55.14,
 (1, 5): None,
 (2, 1): 30.05,
 (2, 2): 36.46,
 (2, 3): 42.29,
 (2, 4): 66.66,
 (2, 5): 76.21,
 (3, 1): 33.82,
 (3, 2): 30.2,
 (3, 3): -500,
 (3, 4): 76.68,
 (3, 5): 87.38,
 (4, 1): 39.43,
 (4, 2): 48.04,
 (4, 3): 55.84,
 (4, 4): 86.69,
 (4, 5): 100,
 (5, 1): None,
 (5, 2): 54.39,
 (5, 3): 62.69,
 (5, 4): 72.04,
 (5, 5): None}

In [23]:
u_tba, v = visualise_table(5,5,u100_4_1)
print(u_tba)
print(v)

[['X', 76.21, 87.38, 100, 'X'], [55.14, 66.66, 76.68, 86.69, 72.04], [47.71, 42.29, -500, 55.84, 62.69], [41.14, 36.46, 30.2, 48.04, 54.39], ['X', 30.05, 33.82, 39.43, 'X']]
╒═══════╤═══════╤═════════╤════════╤═══════╕
│ X     │ 76.21 │   87.38 │ 100    │ X     │
├───────┼───────┼─────────┼────────┼───────┤
│ 55.14 │ 66.66 │   76.68 │  86.69 │ 72.04 │
├───────┼───────┼─────────┼────────┼───────┤
│ 47.71 │ 42.29 │ -500    │  55.84 │ 62.69 │
├───────┼───────┼─────────┼────────┼───────┤
│ 41.14 │ 36.46 │   30.2  │  48.04 │ 54.39 │
├───────┼───────┼─────────┼────────┼───────┤
│ X     │ 30.05 │   33.82 │  39.43 │ X     │
╘═══════╧═══════╧═════════╧════════╧═══════╛


In [24]:
policy_4_1 = visualise_policy(u_tba, D, H, obstacle)
print(policy_4_1)

╒═══╤═══╤══════╤═════╤═══╕
│ X │ → │ →    │ 100 │ X │
├───┼───┼──────┼─────┼───┤
│ → │ → │ ↑    │ ↑   │ ← │
├───┼───┼──────┼─────┼───┤
│ ↑ │ ↑ │ -500 │ ↑   │ ↑ │
├───┼───┼──────┼─────┼───┤
│ ↑ │ ↑ │ →    │ ↑   │ ↑ │
├───┼───┼──────┼─────┼───┤
│ X │ ↑ │ →    │ ↑   │ X │
╘═══╧═══╧══════╧═════╧═══╛


Using R1 and R2 from policy 2

In [25]:
r = -1 #live-in reward
R1 = 50 #exit reward
R2 = -50 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U_4_2 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new4_2 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward_4_2 = {(col, row):r for col in range(1,6) for row in range(1,6)}
U_4_2[D]= R1
U_4_2[H]= R2
U_new4_2[D]= R1
U_new4_2[H]= R2
reward_4_2[D]= R1
reward_4_2[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U_4_2[i] = None
  U_new4_2[i] = None
  reward_4_2[i] = None
print(U_4_2)
print(U_new4_2)
print(reward_4_2)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -50, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 50, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -50, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 50, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -1, (1, 3): -1, (1, 4): -1, (1, 5): None, (2, 1): -1, (2, 2): -1, (2, 3): -1, (2, 4): -1, (2, 5): -1, (3, 1): -1, (3, 2): -1, (3, 3): -50, (3, 4): -1, (3, 5): -1, (4, 1): -1, (4, 2): -1, (4, 3): -1, (4, 4): -1, (4, 5): 50, (5, 1): None, (5, 2): -1, (5, 3): -1, (5, 4): -1, (5, 5): None}


In [26]:
u100_4_2 = valueIteration(0.01, 100, 0.9,reward_4_2, U_4_2, obstacle, U_new4_2, 5, 5,D,H)
u100_4_2

At what iteration it's stopping 15


{(1, 1): None,
 (1, 2): 18.54,
 (1, 3): 21.99,
 (1, 4): 25.85,
 (1, 5): None,
 (2, 1): 15.92,
 (2, 2): 19.85,
 (2, 3): 23.6,
 (2, 4): 31.93,
 (2, 5): 37.03,
 (3, 1): 18.7,
 (3, 2): 19.92,
 (3, 3): -50,
 (3, 4): 37.29,
 (3, 5): 43.12,
 (4, 1): 22.17,
 (4, 2): 27.57,
 (4, 3): 32.73,
 (4, 4): 42.75,
 (4, 5): 50,
 (5, 1): None,
 (5, 2): 25.83,
 (5, 3): 30.16,
 (5, 4): 34.98,
 (5, 5): None}

In [27]:
u_tba, v = visualise_table(5,5,u100_4_2)
print(u_tba)
print(v)

[['X', 37.03, 43.12, 50, 'X'], [25.85, 31.93, 37.29, 42.75, 34.98], [21.99, 23.6, -50, 32.73, 30.16], [18.54, 19.85, 19.92, 27.57, 25.83], ['X', 15.92, 18.7, 22.17, 'X']]
╒═══════╤═══════╤════════╤═══════╤═══════╕
│ X     │ 37.03 │  43.12 │ 50    │ X     │
├───────┼───────┼────────┼───────┼───────┤
│ 25.85 │ 31.93 │  37.29 │ 42.75 │ 34.98 │
├───────┼───────┼────────┼───────┼───────┤
│ 21.99 │ 23.6  │ -50    │ 32.73 │ 30.16 │
├───────┼───────┼────────┼───────┼───────┤
│ 18.54 │ 19.85 │  19.92 │ 27.57 │ 25.83 │
├───────┼───────┼────────┼───────┼───────┤
│ X     │ 15.92 │  18.7  │ 22.17 │ X     │
╘═══════╧═══════╧════════╧═══════╧═══════╛


In [28]:
policy_4_2 = visualise_policy(u_tba, D, H, obstacle)
print(policy_4_2)

╒═══╤═══╤═════╤════╤═══╕
│ X │ → │ →   │ 50 │ X │
├───┼───┼─────┼────┼───┤
│ → │ → │ ↑   │ ↑  │ ← │
├───┼───┼─────┼────┼───┤
│ ↑ │ ↑ │ -50 │ ↑  │ ↑ │
├───┼───┼─────┼────┼───┤
│ ↑ │ ↑ │ →   │ ↑  │ ↑ │
├───┼───┼─────┼────┼───┤
│ X │ ↑ │ →   │ ↑  │ X │
╘═══╧═══╧═════╧════╧═══╛


If we use r=-1 for policy 1 R1 and R2 values, ran the below code.

In [29]:
r = -1 #live-in reward
R1 = 10 #exit reward
R2 = -5 #hazard reward
D, H = (4,5), (3,3) #destination, hazard
U_5 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
U_new5 = {(col, row):0 for col in range(1,6) for row in range(1,6)}
reward_5 = {(col, row):r for col in range(1,6) for row in range(1,6)}
U_5[D]= R1
U_5[H]= R2
U_new5[D]= R1
U_new5[H]= R2
reward_5[D]= R1
reward_5[H]= R2
obstacle = ((1,1),(1,5),(5,1),(5,5))
for i in obstacle:
  U_5[i] = None
  U_new5[i] = None
  reward_5[i] = None
print(U_5)
print(U_new5)
print(reward_5)

{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -5, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 10, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): 0, (1, 3): 0, (1, 4): 0, (1, 5): None, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (3, 1): 0, (3, 2): 0, (3, 3): -5, (3, 4): 0, (3, 5): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0, (4, 5): 10, (5, 1): None, (5, 2): 0, (5, 3): 0, (5, 4): 0, (5, 5): None}
{(1, 1): None, (1, 2): -1, (1, 3): -1, (1, 4): -1, (1, 5): None, (2, 1): -1, (2, 2): -1, (2, 3): -1, (2, 4): -1, (2, 5): -1, (3, 1): -1, (3, 2): -1, (3, 3): -5, (3, 4): -1, (3, 5): -1, (4, 1): -1, (4, 2): -1, (4, 3): -1, (4, 4): -1, (4, 5): 10, (5, 1): None, (5, 2): -1, (5, 3): -1, (5, 4): -1, (5, 5): None}


In [30]:
u100_5 = valueIteration(0.01, 100, 0.9,reward_5, U_5, obstacle, U_new5, 5, 5,D,H)
u100_5

At what iteration it's stopping 14


{(1, 1): None,
 (1, 2): -0.16,
 (1, 3): 1.0,
 (1, 4): 2.3,
 (1, 5): None,
 (2, 1): -0.38,
 (2, 2): 0.74,
 (2, 3): 2.08,
 (2, 4): 4.02,
 (2, 5): 5.68,
 (3, 1): 0.46,
 (3, 2): 1.47,
 (3, 3): -5,
 (3, 4): 5.77,
 (3, 5): 7.71,
 (4, 1): 1.69,
 (4, 2): 3.3,
 (4, 3): 5.1,
 (4, 4): 7.6,
 (4, 5): 10,
 (5, 1): None,
 (5, 2): 2.25,
 (5, 3): 3.71,
 (5, 4): 5.32,
 (5, 5): None}

In [31]:
u_tba, v = visualise_table(5,5,u100_5)
print(u_tba)
print(v)

[['X', 5.68, 7.71, 10, 'X'], [2.3, 4.02, 5.77, 7.6, 5.32], [1.0, 2.08, -5, 5.1, 3.71], [-0.16, 0.74, 1.47, 3.3, 2.25], ['X', -0.38, 0.46, 1.69, 'X']]
╒═══════╤═══════╤═══════╤═══════╤══════╕
│ X     │  5.68 │  7.71 │ 10    │ X    │
├───────┼───────┼───────┼───────┼──────┤
│ 2.3   │  4.02 │  5.77 │  7.6  │ 5.32 │
├───────┼───────┼───────┼───────┼──────┤
│ 1.0   │  2.08 │ -5    │  5.1  │ 3.71 │
├───────┼───────┼───────┼───────┼──────┤
│ -0.16 │  0.74 │  1.47 │  3.3  │ 2.25 │
├───────┼───────┼───────┼───────┼──────┤
│ X     │ -0.38 │  0.46 │  1.69 │ X    │
╘═══════╧═══════╧═══════╧═══════╧══════╛


In [32]:
policy_new = visualise_policy(u_tba, D, H, obstacle)
print(policy_new)

╒═══╤═══╤════╤════╤═══╕
│ X │ → │ →  │ 10 │ X │
├───┼───┼────┼────┼───┤
│ → │ → │ ↑  │ ↑  │ ← │
├───┼───┼────┼────┼───┤
│ ↑ │ ↑ │ -5 │ ↑  │ ↑ │
├───┼───┼────┼────┼───┤
│ ↑ │ ↑ │ →  │ ↑  │ ↑ │
├───┼───┼────┼────┼───┤
│ X │ ↑ │ →  │ ↑  │ X │
╘═══╧═══╧════╧════╧═══╛
